<a href="https://colab.research.google.com/github/harjeet88/A_For_Algorithms/blob/master/stocks/stock_prediction_strategy_grok_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install yfinance --upgrade

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import yfinance as yf
import matplotlib.pyplot as plt

In [81]:
# Simulated portfolio parameters
initial_capital = 1000000  # ₹10,00,000 initial investment
target_annual_return = 0.25  # 25% annualized return
equity_allocation = 0.80  # 80% in equities
hedge_allocation = 0.10  # 10% in gold ETFs
cash_allocation = 0.10  # 10% in liquid funds
max_position_size = 0.05  # 5% per stock
stop_loss_pct = 0.15  # 15% trailing stop-loss
volatility_threshold = 0.40  # 40% max volatility
rebalance_freq = 'Q'  # Quarterly rebalancing
transaction_cost = 0.001  # 0.1% per trade

In [56]:
# High-conviction stock watchlist (NSE tickers)
stocks = [
    'RELIANCE.NS',  # Large-cap: Diversified, PLI beneficiary
    'TATAMOTORS.NS',  # Mid-cap: Auto, EV focus
#    'DIXON.NS',  # Mid-cap: Electronics manufacturing (PLI)
#    'POLYCAB.NS',  # Mid-cap: Electricals, renewable energy
    'CDSL.NS',  # Small-cap: Fintech, capital market
#    'NAVINFLUOR.NS',  # Small-cap: Specialty chemicals
#    'KPITTECH.NS',  # Small-cap: Auto-tech, EV software
#    'L&TFH.NS',  # Mid-cap: Affordable housing finance
#    'GREENPOWER.NS',  # Small-cap: Renewable energy
#    'BIRLACORPN.NS'  # Mid-cap: Cement, affordable housing
]

In [57]:
# Sector allocation weights (70% mid/small-cap, 30% large-cap)
sector_weights = {
    'Renewables': 0.30,  # e.g., GREENPOWER.NS, POLYCAB.NS
    'Specialty Chemicals': 0.25,  # e.g., NAVINFLUOR.NS
    'Fintech': 0.20,  # e.g., CDSL.NS, KPITTECH.NS
    'Manufacturing (PLI)': 0.15,  # e.g., DIXON.NS, RELIANCE.NS
    'Affordable Housing': 0.10  # e.g., L&TFH.NS, BIRLACORPN.NS
}

In [77]:
# Backtesting period (2018-2023)
start_date = '2024-01-01'
end_date = '2024-12-31'

In [68]:
def fetch_data(tickers, start, end):
    data = {}
    failed_tickers = []
    for ticker in tickers:
        try:
            df = yf.download(ticker, start=start, end=end, progress=False, auto_adjust=False)
            if not df.empty and 'Adj Close' in df.columns and len(df) >= 2:
                data[ticker] = df['Adj Close'] # Store the Series directly
            else:
                failed_tickers.append(ticker)
                print(f"No valid data for {ticker}: Insufficient data points or missing 'Adj Close'")
        except Exception as e:
            failed_tickers.append(ticker)
            print(f"Error fetching data for {ticker}: {e}")
    if not data:
        print("No data fetched for any tickers. Exiting.")
        return pd.DataFrame()
    if failed_tickers:
        print(f"Failed to fetch data for: {failed_tickers}")
    # Ensure DataFrame has a proper index by using pd.concat with the Series
    try:
        # Concatenate the Series into a DataFrame
        return pd.concat(data.values(), axis=1, keys=data.keys())
    except ValueError as e:
        print(f"Error creating DataFrame: {e}. Likely scalar values detected.")
        return pd.DataFrame()

In [60]:
# Calculate returns and volatility
def calculate_metrics(df):
    returns = df.pct_change().dropna()
    annual_returns = returns.mean() * 252  # Annualized return
    annual_vol = returns.std() * np.sqrt(252)  # Annualized volatility
    sharpe_ratio = annual_returns / annual_vol  # Assuming risk-free rate = 0 for simplicity
    return annual_returns, annual_vol, sharpe_ratio

In [79]:
# Simulate portfolio with rebalancing and stop-loss
def simulate_portfolio(df, weights, initial_capital):
    if df.empty or len(df) < 2:
        print("Insufficient data for simulation.")
        return 0, 0, 0, pd.Series(dtype=float)

    portfolio_value = initial_capital
    positions = {stock: initial_capital * weights.get(stock, max_position_size) / df[stock].iloc[0] for stock in df.columns}
    portfolio_values = []
    stop_loss_triggered = {stock: False for stock in df.columns}
    valid_dates = df.index[1:]
    peak_prices = {stock: df[stock].iloc[0] for stock in df.columns}  # Track peak for trailing stop-loss

    if len(valid_dates) == 0:
        print("No valid dates for simulation.")
        return 0, 0, 0, pd.Series(dtype=float)

    for i, date in enumerate(valid_dates):
        daily_value = 0
        for stock in df.columns:
            if not stop_loss_triggered[stock]:
                price = df[stock].loc[date]
                peak_prices[stock] = max(peak_prices[stock], price)  # Update peak
                if peak_prices[stock] > 0 and (peak_prices[stock] - price) / peak_prices[stock] > stop_loss_pct:
                    stop_loss_triggered[stock] = True
                    positions[stock] = 0  # Sell position
                    portfolio_value -= transaction_cost * daily_value  # Apply transaction cost
                daily_value += positions[stock] * price
        portfolio_values.append(daily_value if daily_value > 0 else portfolio_value)
        portfolio_value = daily_value

        # Quarterly rebalancing
        if i > 0 and date.quarter != valid_dates[i-1].quarter:
            total_value = sum(positions[stock] * df[stock].loc[date] for stock in df.columns if not stop_loss_triggered[stock])
            if total_value > 0:
                positions = {stock: (total_value * weights.get(stock, max_position_size) / df[stock].loc[date]) for stock in df.columns}
                stop_loss_triggered = {stock: False for stock in df.columns}
                portfolio_value -= transaction_cost * total_value  # Apply transaction cost
                peak_prices = {stock: df[stock].loc[date] for stock in df.columns}  # Reset peaks

    try:
        portfolio_df = pd.Series(portfolio_values, index=valid_dates, dtype=float)
    except ValueError as e:
        print(f"Error creating portfolio Series: {e}")
        return 0, 0, 0, pd.Series(dtype=float)

    if portfolio_df.empty or portfolio_df.iloc[-1] <= 0:
        print("Portfolio DataFrame is empty or portfolio value reached zero.")
        return 0, 0, 0, pd.Series(dtype=float)

    returns = portfolio_df.pct_change().dropna()
    cagr = ((portfolio_df.iloc[-1] / initial_capital) ** (1 / 5)) - 1 if not portfolio_df.empty else 0
    sharpe = returns.mean() * 252 / (returns.std() * np.sqrt(252)) if not returns.empty and returns.std() != 0 else 0
    max_drawdown = (portfolio_df / portfolio_df.cummax() - 1).min() if not portfolio_df.empty else 0
    return cagr, sharpe, max_drawdown, portfolio_df

In [73]:
# Main execution
def run():
    df = fetch_data(stocks, start_date, end_date)
    #print(df)
    if df.empty:
        print("No data available to proceed. Please check ticker symbols or data source.")
        return

    # Assign weights (70% mid/small-cap, 30% large-cap)
    print("fetched all data")
    weights = {}
    large_caps = ['RELIANCE.NS']
    print(len(large_caps))
    mid_small_caps = [s for s in df.columns if s not in large_caps]

    print(len(mid_small_caps))
    for stock in large_caps:
        print("in large caps")
        if stock in df.columns:
            weights[stock] = 0.30 / len([s for s in large_caps if s in df.columns])
    for stock in mid_small_caps:
        print("in mid small caps")
        sector = next((k for k, v in sector_weights.items() if stock in [s for s in stocks if k in s]), 'Other')
        weights[stock] = 0.70 * sector_weights.get(sector, 0.2) / len(mid_small_caps)

    print("before simulate_portfolio")

    # Simulate portfolio
    cagr, sharpe, max_drawdown, portfolio_df = simulate_portfolio(df, weights, initial_capital)

    # Print results
    print(f"CAGR: {cagr:.2%}")
    print(f"Sharpe Ratio: {sharpe:.2f}")
    print(f"Max Drawdown: {max_drawdown:.2%}")

    # Plot portfolio value
    if not portfolio_df.empty:
        plt.figure(figsize=(10, 6))
        portfolio_df.plot(title='Portfolio Value (2018-2023)')
        plt.xlabel('Date')
        plt.ylabel('Portfolio Value (₹)')
        plt.grid(True)
        plt.show()


In [82]:
run()

fetched all data
1
1
in large caps
in mid small caps
before simulate_portfolio
Portfolio DataFrame is empty or portfolio value reached zero.
CAGR: 0.00%
Sharpe Ratio: 0.00
Max Drawdown: 0.00%


In [75]:
stocks =['RELIANCE.NS']
fetch_data(stocks, start_date, end_date)

,RELIANCE.NS
Ticker,RELIANCE.NS
Date,
2018-01-01,413.114288
2018-01-02,413.750000
2018-01-03,415.407471
2018-01-04,417.904999
2018-01-05,419.244598
...,...
2023-12-22,1282.525024
2023-12-26,1289.025024


In [ ]:
if __name__ == "__main__":
    run()